In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

### LSTM이 제공하는 features를 사용하는 CRF 모델

모든 가능한 state의 집합을 S, 인풋 시퀀스를 x라고 할 때, CRF는 아래와 같은 모델을 만드는 것
<br>
$$p(s_1 . . . s_m|x_1 . . . x_m) = p(s|x)$$<br>
CRF의 중요 아이디어 중 하나는 feature function을 정의하는 것이다. <br>
$$Φ(x, s) ∈ R^d$$ <br>
feature function은 전체 인풋 시퀀스 x를 (state 시퀀스 s, d차원 feature vector)로 매핑한다

* Write the recurrence for the viterbi variable at step i for tag k.
* Modify the above recurrence to compute the forward variables instead.
* Modify again the above recurrence to compute the forward variables in log-space (hint: log-sum-exp)

$$ P(y|x) = \frac{\exp{(\text{Score}(x, y)})}{\sum_{y'} \exp{(\text{Score}(x, y')})} $$

y가 태그 시퀀스, x가 단어들의 인풋 시퀀스라고 했을 때, 위 식을 계산한다<br>
Where the score is determined by defining some log potentials $\log \psi_i(x,y)$ such that $$ \text{Score}(x,y) = \sum_i \log \psi_i(x,y) $$

### emission and trainstion

emission potential for the word at index i는 Bi-LSTM의 timestep i번째 히든 스테이트로부터 온다. transition score는 T x T 크기의 매트릭스 P에 저장된다. (T는 tagset) <br><br>
$$ \text{Score}(x,y) = \sum_i \log \psi_\text{EMIT}(y_i \rightarrow x_i) + \log \psi_\text{TRANS}(y_{i-1} \rightarrow y_i) $$$$ = \sum_i h_i[y_i] + \textbf{P}_{y_i, y_{i-1}} $$

### 헬퍼 펑션 

In [3]:
# Helper functions to make the code more readable.
def prepare_sequence(seq, to_ix):
    idxs = list(map(lambda w: to_ix[w], seq))
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

def to_scalar(var):
    # returns a python float
    return var.view(-1).data.tolist()[0]

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return to_scalar(idx)

# Compute log sum exp in a numerically stable way for the forward algorithm
# Forward 알고리즘을 수치적으로 안정시키기 위해 곱셈을 log 취해서 Sum 한다.
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

### 모델링 

The partition function is an integral (for continuous variables) or sum (for discretevariables) over the unnormalized probability of all states

$$ \alpha_i(j) = \sum_{j' \in T} \psi_\text{EMIT}(j \rightarrow i) \times \psi_\text{TRANS}(j' \rightarrow j) \times \alpha_{i-1}(j') $$
$$ \log \alpha_i(j) = \log \overbrace{\sum_{j' \in T} \exp{(\log \psi_\text{EMIT}(j \rightarrow i) + \log \psi_\text{TRANS}(j' \rightarrow j) + \log \alpha_{i-1}(j'))}}^\text{transform out of log-space and compute forward variable} $$

#### function ex

In [5]:
torch.Tensor(1, 5).fill_(-10000.)


-10000 -10000 -10000 -10000 -10000
[torch.FloatTensor of size 1x5]

In [8]:
x = torch.Tensor([[1], [2], [3]])
print(x.size())
print(x)
print(x.expand(3, 4))

torch.Size([3, 1])

 1
 2
 3
[torch.FloatTensor of size 3x1]


 1  1  1  1
 2  2  2  2
 3  3  3  3
[torch.FloatTensor of size 3x4]



In [9]:
class BiLSTM_CRF(nn.Module):
    
    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim//2, num_layers=1, bidirectional=True)
        # forward 와 backward hidden vector를 concat하는듯?
        
        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        
        # 트렌지션 매트릭스 
        # Matrix of transition parameters.  Entry i,j is the score of transitioning *to* i *from* j.
        self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))
        
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # bi-direc이니까 히든스테이트도 2개씩
        return ( autograd.Variable( torch.randn(2, 1, self.hidden_dim)),
                 autograd.Variable( torch.randn(2, 1, self.hidden_dim)) )
    
    
    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.Tensor(1, self.tagset_size).fill_(-10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0. # 각 태그 마다의 점수를 저장하는 건가
        
        # Wrap in a variable so that we will get automatic backprop
        forward_var = autograd.Variable(init_alphas)
        
        # Iterate through the sentence
        for feat in feats: #문장의 피처들
            alphas_t = [] # The forward variables at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of the previous tag
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the edge (i -> next_tag)
                # before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the scores.
                alphas_t.append(log_sum_exp(next_tag_var))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
        
    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats
        
    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = autograd.Variable( torch.Tensor([0]) )
        tags = torch.cat( [torch.LongTensor([self.tag_to_ix[START_TAG]]), tags] )
        for i, feat in enumerate(feats):
            score = score + self.transitions[tags[i+1], tags[i]] + feat[tags[i+1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score
    
    def _viterbi_decode(self, feats):
        backpointers = []
        
        # Initialize the viterbi variables in log space
        init_vvars = torch.Tensor(1, self.tagset_size).fill_(-10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0
        
        # forward_var at step i holds the viterbi variables for step i-1 
        forward_var = autograd.Variable(init_vvars)
        for feat in feats:
            bptrs_t = [] # holds the backpointers for this step
            viterbivars_t = [] # holds the viterbi variables for this step
            
            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the previous step,
                # plus the score of transitioning from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id])
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)
        
        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]
        
        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG] # Sanity check
        best_path.reverse()
        return path_score, best_path
        
    def neg_log_likelihood(self, sentence, tags):
        self.hidden = self.init_hidden()
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score
        
    def forward(self, sentence): # dont confuse this with _forward_alg above.
        self.hidden = self.init_hidden()
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)
        
        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

### 데이터 준비 

### 토이

In [10]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# Make up some training data
training_data = [ (
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
) ]

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            
tag_to_ix = { "B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4 }

### 리얼 with Batch 

In [15]:
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

In [16]:
pre_train_x = [t[0] for t in training_data]
train_y = [t[1] for t in training_data]

In [93]:
train_x=[]

In [94]:
for tr in pre_train_x:
    temp = prepare_sequence(tr, word_to_ix)
    train_x.append((temp,len(temp.data.numpy())))

In [107]:
x = torch.cat(X).view(, -1) 

RuntimeError: size '[10 x 32 x -1]' is invalid for input of with 347 elements at /b/wheel/pytorch-src/torch/lib/TH/THStorage.c:55

In [47]:
X,lens = zip(*sorted(train_x[:32],key=lambda x:x[1],reverse=True))


lens = [10,8,5,4,3]

px = pack_padded_sequence(x, lens, batch_first=True) # lens는 decreasing order로 줘야 함

In [20]:
import json

In [21]:
train = json.load(open('../../dataset/NER_16000_dev.json'))

In [29]:
training_data=[]

for sent in train:
    word=[]
    tag=[]
    for w,p,t in sent:
        word.append(w)
        tag.append(t)
    training_data.append((word,tag))

In [30]:
NER_LIST = ['B-PER','I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG','B-DATE', 'I-DATE','B-TIME','I-TIME','B-MISC','I-MISC','O',"<START>","<STOP>"]
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 30
HIDDEN_DIM = 20

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

ix_to_word = {v:k for k,v in word_to_ix.items()}

tag_to_ix={}
i=0
for tag in NER_LIST:           
    tag_to_ix[tag] = i
    i+=1

In [11]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

In [12]:
# Check predictions before training
precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
precheck_tags = torch.LongTensor([ tag_to_ix[t] for t in training_data[0][1] ])
print(model(precheck_sent))

(Variable containing:
 12.7272
[torch.FloatTensor of size 1]
, [2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 2])


In [13]:
%%time
# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(300): # again, normally you would NOT do 300 epochs, it is toy data
    
    print(epoch)
    
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.  We need to clear them out
        # before each instance
        model.zero_grad()
    
        # Step 2. Get our inputs ready for the network, that is, turn them into Variables
        # of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.LongTensor([ tag_to_ix[t] for t in tags ])
    
        # Step 3. Run our forward pass.
        neg_log_likelihood = model.neg_log_likelihood(sentence_in, targets)
    
        # Step 4. Compute the loss, gradients, and update the parameters by calling
        # optimizer.step()
        neg_log_likelihood.backward()
        optimizer.step()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [14]:
# Check predictions after training
precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
print(model(precheck_sent))
# We got it!

(Variable containing:
 36.6214
[torch.FloatTensor of size 1]
, [0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 2])
